In [1]:
# Parameters
credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"
table_name = "raw_data_cleaned.humanresources_employee"


### EDA (Exploratory Data Analysis) & Data Cleaning

In [1]:

##import das bibliotecas e adequando colunas, linhas e formato de números

from google.cloud import bigquery
from dotenv import load_dotenv
import pandas as pd
import pandas_gbq as gbq
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re

# Carrega o .env
load_dotenv()

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

# Verificar se a configuração está correta
print("Credenciais do BigQuery:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

# Verifica se a variável está configurada
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))


# Configurar Pandas para exibir todas as colunas e todas as linhas completas
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


pd.options.display.float_format = '{:.2f}'.format

Credenciais do BigQuery: /mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json
/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json


In [2]:
# Configurar o cliente do BigQuery
client = bigquery.Client()

# Nome do dataset e tabela
dataset_id = 'raw_data'

DefaultCredentialsError: File /mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json was not found.

In [4]:
# Listar tabelas no dataset
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)




Tabelas disponíveis:


humanresources_employee
person_address
person_businessentity
person_person
production_location
production_product
production_productcategory
production_productinventory
production_productsubcategory
sales_customer
sales_salesorderdetail
sales_salesorderheader
sales_salesterritory
sales_store


In [5]:
query = f"SELECT * FROM `raw_data.humanresources_employee`"
data = client.query(query).result().to_dataframe()

# Expandir a coluna JSON
EDA_humanresources_employee = pd.json_normalize(data['data'])

# Exibir os dados expandidos
print(EDA_humanresources_employee.sample(n=50))


/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/airflow_env/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/airflow_env/lib/python3.10/site-packages/google/cloud/bigquery/_pandas_helpers.py:208: UserWarning: Unable to determine Arrow type for field 'data'.
  warnings.warn(


       birthdate  businessentityid  currentflag gender    hiredate                      jobtitle                     loginid maritalstatus                modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
1453  1988-05-10               204         True      M  2009-03-08  Production Technician - WC40       adventure-works\gabe0             M         2014-06-30T00:00:00        440379437      /3/1/21/12/  790c9a7c-264c-4d48-a9d3-e2571a8011d8         False              48             57
41    1974-09-10               141         True      M  2010-01-24  Production Technician - WC20        adventure-works\min0             M         2014-06-30T00:00:00        872923042       /3/1/14/7/  cc75c35d-1566-43fa-b2e9-d1e508efdcd1         False              20              1
1218  1977-10-04               197         True      M  2008-12-31  Production Technician - WC40     adventure-works\rajesh0             M         2

### Verificando valores nulos/em branco nos dados

In [6]:
#valores nulos

EDA_humanresources_employee.isnull().sum()

birthdate           0
businessentityid    0
currentflag         0
gender              0
hiredate            0
jobtitle            0
loginid             0
maritalstatus       0
modifieddate        0
nationalidnumber    0
organizationnode    0
rowguid             0
salariedflag        0
sickleavehours      0
vacationhours       0
dtype: int64

In [7]:
# dropar colunas vazias

In [8]:
#total valores unicos de cada variável

valores_unicos = []

for i in EDA_humanresources_employee.columns[0:15].tolist():
    print(i, ':', len(EDA_humanresources_employee[i].astype(str).value_counts()))
    valores_unicos.append(len(EDA_humanresources_employee[i].astype(str).value_counts()))

birthdate : 275
businessentityid : 290
currentflag : 1
gender : 2
hiredate : 164
jobtitle : 67
loginid : 290
maritalstatus : 2
modifieddate : 2
nationalidnumber : 290
organizationnode : 290
rowguid : 290
salariedflag : 2
sickleavehours : 51
vacationhours : 100


In [9]:
# Identificar duplicatas com base em 'businessentityid'
duplicatas = EDA_humanresources_employee[EDA_humanresources_employee.duplicated(subset=['businessentityid'], keep=False)]

# Verificar se existem duplicatas
if not duplicatas.empty:
    # Ordenar duplicatas por 'businessentityid' e 'modifieddate'
    duplicatas_ordenadas = duplicatas.sort_values(by=['businessentityid', 'modifieddate'])
    
    # Exibir duplicatas ordenadas
    print("Duplicatas ordenadas:")
    print(duplicatas_ordenadas)
else:
    print("Não foram encontradas duplicatas.")


Duplicatas ordenadas:
       birthdate  businessentityid  currentflag gender    hiredate                 jobtitle                  loginid maritalstatus         modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
2583  1969-01-29                 1         True      M  2009-01-14  Chief Executive Officer     adventure-works\ken0             S  2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
2584  1969-01-29                 1         True      M  2009-01-14  Chief Executive Officer     adventure-works\ken0             S  2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
2585  1969-01-29                 1         True      M  2009-01-14  Chief Executive Officer     adventure-works\ken0             S  2014-06-30T00:00:00        295847284   

In [10]:
print(duplicatas_ordenadas.drop_duplicates())


       birthdate  businessentityid  currentflag gender    hiredate                       jobtitle                   loginid maritalstatus         modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
2583  1969-01-29                 1         True      M  2009-01-14        Chief Executive Officer      adventure-works\ken0             S  2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
27    1971-08-01                 2         True      F  2008-01-31  Vice President of Engineering    adventure-works\terri0             S  2014-06-30T00:00:00        245797967              /1/  45e8f437-670d-4409-93cb-f9424a40d6ee          True              20              1
54    1974-11-12                 3         True      M  2007-11-11            Engineering Manager  adventure-works\roberto0             M  2014-06-30T00:00:00        509647

In [11]:
duplicados_businessentityid = EDA_humanresources_employee[EDA_humanresources_employee.duplicated(subset=['businessentityid'], keep=False)]

# Ordenar por 'businessentityid' para facilitar a análise
duplicados_ordenados = duplicados_businessentityid.sort_values(by=['businessentityid'])

# Exibir todas as linhas duplicadas
print(duplicados_ordenados)


       birthdate  businessentityid  currentflag gender    hiredate                 jobtitle                  loginid maritalstatus         modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
2591  1969-01-29                 1         True      M  2009-01-14  Chief Executive Officer     adventure-works\ken0             S  2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
2590  1969-01-29                 1         True      M  2009-01-14  Chief Executive Officer     adventure-works\ken0             S  2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
2587  1969-01-29                 1         True      M  2009-01-14  Chief Executive Officer     adventure-works\ken0             S  2014-06-30T00:00:00        295847284                /  f01251

In [12]:
pd.set_option('display.max_rows', None)
contagem = EDA_humanresources_employee['businessentityid'].value_counts()

# Filtrar apenas os IDs que aparecem mais de uma vez
repetidos = contagem[contagem > 1]

# Exibir repetidos novamente
print(repetidos)

businessentityid
88     9
139    9
227    9
209    9
53     9
233    9
118    9
49     9
230    9
120    9
206    9
50     9
214    9
207    9
51     9
213    9
208    9
52     9
62     9
212    9
40     9
190    9
216    9
210    9
48     9
215    9
219    9
187    9
55     9
220    9
27     9
188    9
211    9
189    9
287    9
35     9
172    9
280    9
32     9
175    9
283    9
38     9
184    9
29     9
185    9
36     9
181    9
285    9
28     9
182    9
34     9
153    9
10     9
37     9
155    9
33     9
183    9
31     9
58     9
157    9
59     9
159    9
60     9
30     9
158    9
274    9
278    9
168    9
43     9
290    9
150    9
41     9
288    9
146    9
63     9
169    9
69     9
282    9
171    9
67     9
173    9
70     9
68     9
276    9
174    9
66     9
176    9
65     9
279    9
179    9
177    9
277    9
39     9
167    9
170    9
64     9
281    9
178    9
75     9
222    9
22     9
76     9
224    9
23     9
77     9
223    9
72     9
17     9
180    9
73

In [13]:
#copia da humanresources_employee
raw_data_bkp = EDA_humanresources_employee.copy()

# Ordenar o DataFrame por 'businessentityid' e 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.sort_values(by=['businessentityid', 'modifieddate'])

# Remover duplicatas mantendo a última ocorrência com base em 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicatas (baseando-se na última 'modifieddate'): {len(EDA_humanresources_employee)}")


Linhas após remover duplicatas (baseando-se na última 'modifieddate'): 290


In [14]:
# Verificar informações do DataFrame
print(EDA_humanresources_employee.info())


<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, 2591 to 845
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   birthdate         290 non-null    object
 1   businessentityid  290 non-null    int64 
 2   currentflag       290 non-null    bool  
 3   gender            290 non-null    object
 4   hiredate          290 non-null    object
 5   jobtitle          290 non-null    object
 6   loginid           290 non-null    object
 7   maritalstatus     290 non-null    object
 8   modifieddate      290 non-null    object
 9   nationalidnumber  290 non-null    object
 10  organizationnode  290 non-null    object
 11  rowguid           290 non-null    object
 12  salariedflag      290 non-null    bool  
 13  sickleavehours    290 non-null    int64 
 14  vacationhours     290 non-null    int64 
dtypes: bool(2), int64(3), object(10)
memory usage: 32.3+ KB
None


In [15]:
# Identificar colunas com datas
date_columns = ['birthdate', 'hiredate', 'modifieddate']

# Converter as colunas para datetime se ainda não estiverem
for col in date_columns:
    EDA_humanresources_employee[col] = pd.to_datetime(EDA_humanresources_employee[col], errors='coerce')

# Criar uma cópia do DataFrame para exibição formatada
formatted_data = EDA_humanresources_employee.copy()

# Formatar todas as colunas de datas para exibição
for col in date_columns:
    formatted_data[col] = EDA_humanresources_employee[col].dt.strftime('%Y-%m-%d %H:%M:%S')

# Exibir o DataFrame formatado
print(formatted_data.head())

# Verificar os tipos originais permanecem datetime64[ns]
print("\nTipos originais das colunas no DataFrame principal:")
print(EDA_humanresources_employee[date_columns].dtypes)


                birthdate  businessentityid  currentflag gender             hiredate                       jobtitle                   loginid maritalstatus         modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
2591  1969-01-29 00:00:00                 1         True      M  2009-01-14 00:00:00        Chief Executive Officer      adventure-works\ken0             S  2014-06-30 00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
35    1971-08-01 00:00:00                 2         True      F  2008-01-31 00:00:00  Vice President of Engineering    adventure-works\terri0             S  2014-06-30 00:00:00        245797967              /1/  45e8f437-670d-4409-93cb-f9424a40d6ee          True              20              1
62    1974-11-12 00:00:00                 3         True      M  2007-11-11 00:00:00            Engineering Manager  a

In [16]:
# Padronizar textos em jobtitle e gender
EDA_humanresources_employee['jobtitle'] = EDA_humanresources_employee['jobtitle'].str.strip().str.title()
EDA_humanresources_employee['gender'] = EDA_humanresources_employee['gender'].str.strip().str.upper()

# Verificar valores únicos para garantir a padronização
print("Valores únicos em 'jobtitle':", EDA_humanresources_employee['jobtitle'].unique())
print("Valores únicos em 'gender':", EDA_humanresources_employee['gender'].unique())


Valores únicos em 'jobtitle': ['Chief Executive Officer' 'Vice President Of Engineering'
 'Engineering Manager' 'Senior Tool Designer' 'Design Engineer'
 'Research And Development Manager' 'Research And Development Engineer'
 'Tool Designer' 'Senior Design Engineer' 'Marketing Manager'
 'Marketing Assistant' 'Marketing Specialist'
 'Vice President Of Production' 'Production Control Manager'
 'Production Supervisor - Wc60' 'Production Technician - Wc60'
 'Production Supervisor - Wc10' 'Production Technician - Wc10'
 'Production Supervisor - Wc50' 'Production Technician - Wc50'
 'Production Supervisor - Wc30' 'Production Technician - Wc30'
 'Production Supervisor - Wc40' 'Production Technician - Wc40'
 'Shipping And Receiving Supervisor' 'Stocker'
 'Shipping And Receiving Clerk' 'Production Supervisor - Wc20'
 'Production Technician - Wc20' 'Production Supervisor - Wc45'
 'Production Technician - Wc45' 'Quality Assurance Manager'
 'Quality Assurance Supervisor' 'Quality Assurance Technic

In [17]:
# Identificar colunas numéricas para análise 
numeric_columns = ['sickleavehours', 'vacationhours']

# Exibir estatísticas descritivas
print(EDA_humanresources_employee[numeric_columns].describe())

# Calcular limites para outliers (IQR - Intervalo Interquartil)
for col in numeric_columns:
    q1 = EDA_humanresources_employee[col].quantile(0.25)
    q3 = EDA_humanresources_employee[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Filtrar outliers
    outliers = EDA_humanresources_employee[(EDA_humanresources_employee[col] < lower_bound) | (EDA_humanresources_employee[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


       sickleavehours  vacationhours
count          290.00         290.00
mean            45.31          50.61
std             14.54          28.79
min             20.00           0.00
25%             33.00          26.25
50%             46.00          51.00
75%             58.00          75.00
max             80.00          99.00



Coluna: sickleavehours
Limite inferior: -4.5, Limite superior: 95.5
Outliers detectados (0):
Empty DataFrame
Columns: [sickleavehours]
Index: []

Coluna: vacationhours
Limite inferior: -46.875, Limite superior: 148.125
Outliers detectados (0):
Empty DataFrame
Columns: [vacationhours]
Index: []


In [18]:

# Criar a coluna 'hire_year' com base em 'hiredate'
EDA_humanresources_employee['hire_year'] = EDA_humanresources_employee['hiredate'].dt.year


#verificando outros dados para detectar outliers
anos_contratacao = [int(ano) for ano in EDA_humanresources_employee['hire_year'].unique()]
anos_contratacao.sort()
print(anos_contratacao)

print("Valores únicos em 'gender':", EDA_humanresources_employee['gender'].unique())
print("Valores únicos em 'salariedflag':", EDA_humanresources_employee['salariedflag'].unique())


[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]
Valores únicos em 'gender': ['M' 'F']
Valores únicos em 'salariedflag': [ True False]


In [19]:
# Verificar valores ausentes
missing = EDA_humanresources_employee.isnull().sum()
print("Valores ausentes por coluna:")
print(missing)

# Tratar colunas críticas
if missing['hiredate'] > 0:
    print("Tratar valores ausentes em 'hiredate' (decisão: remover ou imputar)")

if missing['jobtitle'] > 0:
    print("Tratar valores ausentes em 'jobtitle' (decisão: remover ou imputar)")

if missing['businessentityid'] > 0:
    print("Erro crítico: 'businessentityid' não pode ter valores ausentes!")


Valores ausentes por coluna:
birthdate           0
businessentityid    0
currentflag         0
gender              0
hiredate            0
jobtitle            0
loginid             0
maritalstatus       0
modifieddate        3
nationalidnumber    0
organizationnode    0
rowguid             0
salariedflag        0
sickleavehours      0
vacationhours       0
hire_year           0
dtype: int64


In [20]:
# Verificar linhas com 'modifieddate' ausente
missing_modifieddate = EDA_humanresources_employee[EDA_humanresources_employee['modifieddate'].isnull()]
print("Linhas com 'modifieddate' ausente:")
print(missing_modifieddate)


Linhas com 'modifieddate' ausente:
      birthdate  businessentityid  currentflag gender   hiredate                    jobtitle                  loginid maritalstatus modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours  hire_year
1079 1982-11-03                26         True      M 2008-12-01  Production Control Manager   adventure-works\peter0             M          NaT        277173473            /3/1/  69d5d162-e817-45e7-9dec-5d9b8310e7b1          True              41             43       2008
2096 1977-10-26               211         True      M 2009-02-28   Quality Assurance Manager   adventure-works\hazem0             S          NaT        398223854            /3/2/  05c84608-f445-4f9d-bb5c-0828c309c29d          True              60             80       2009
1115 1968-09-17               222         True      M 2008-12-12            Master Scheduler  adventure-works\ascott0             S          NaT  

In [21]:
# Preencher 'modifieddate' ausente ou igual a 'hiredate', pois pode ser a ultima data de modificação no sistema.
EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'].isnull() | (EDA_humanresources_employee['modifieddate'] == pd.Timestamp('1900-01-01')), 'modifieddate'] = EDA_humanresources_employee['hiredate']

# Exibir as linhas ajustadas
print("Linhas onde 'modifieddate' foi ajustado para 'hiredate':")
print(EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'] == EDA_humanresources_employee['hiredate']])


Linhas onde 'modifieddate' foi ajustado para 'hiredate':
      birthdate  businessentityid  currentflag gender   hiredate                    jobtitle                  loginid maritalstatus modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours  hire_year
1079 1982-11-03                26         True      M 2008-12-01  Production Control Manager   adventure-works\peter0             M   2008-12-01        277173473            /3/1/  69d5d162-e817-45e7-9dec-5d9b8310e7b1          True              41             43       2008
2096 1977-10-26               211         True      M 2009-02-28   Quality Assurance Manager   adventure-works\hazem0             S   2009-02-28        398223854            /3/2/  05c84608-f445-4f9d-bb5c-0828c309c29d          True              60             80       2009
1115 1968-09-17               222         True      M 2008-12-12            Master Scheduler  adventure-works\ascott0       

In [22]:
# Verificar unicidade de 'businessentityid'
is_unique = EDA_humanresources_employee['businessentityid'].is_unique
print(f"'businessentityid' é único? {is_unique}")


'businessentityid' é único? True


In [23]:
# Definir regex para validar números (exemplo: apenas dígitos, 9 caracteres)
# acrescentei para ver se tinha um padrão, mas não tem
regex = r'^\d{9}$'

# Verificar valores inválidos
invalid_nationalid = EDA_humanresources_employee[~EDA_humanresources_employee['nationalidnumber'].astype(str).str.match(regex)]
print(f"Valores inválidos em 'nationalidnumber':\n{invalid_nationalid['nationalidnumber']}")


Valores inválidos em 'nationalidnumber':
89      42487730
116     56920285
989     24756624
1070    52541318
1124    95958330
1178    72636981
512     14417807
593      6298838
890     66073987
1673    33237992
2276     9659517
287     10708100
242     92096924
719      8066363
791     63179277
809     36151748
1601    58791499
1628     1662732
1736     7201901
2528    90888098
2555    82638150
2510    54759846
1727     1300049
998     45615666
233     63761469
206     25011600
71      56772045
278     97728960
1898    65848458
1763    60114406
800     87268837
1241    19312190
2159    20244403
2312    28414965
1466       30845
1583    60517918
1421    20269531
1934    58317344
557     61161660
980     90836195
Name: nationalidnumber, dtype: object


In [24]:
is_unique = EDA_humanresources_employee['nationalidnumber'].is_unique
print(f"'nationalidnumber' é único? {is_unique}")


'nationalidnumber' é único? True


In [25]:
# Criar um backup do DataFrame tratado
EDA_humanresources_employee_bkp_v2 = EDA_humanresources_employee.copy()

# Verificar o tamanho do backup e as primeiras linhas
print(f"Backup criado com {len(EDA_humanresources_employee_bkp_v2)} linhas.")
print(EDA_humanresources_employee_bkp_v2.head())


Backup criado com 290 linhas.
      birthdate  businessentityid  currentflag gender   hiredate                       jobtitle                   loginid maritalstatus modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours  hire_year
2591 1969-01-29                 1         True      M 2009-01-14        Chief Executive Officer      adventure-works\ken0             S   2014-06-30        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99       2009
35   1971-08-01                 2         True      F 2008-01-31  Vice President Of Engineering    adventure-works\terri0             S   2014-06-30        245797967              /1/  45e8f437-670d-4409-93cb-f9424a40d6ee          True              20              1       2008
62   1974-11-12                 3         True      M 2007-11-11            Engineering Manager  adventure-works\roberto0             M   2

In [26]:
# Verificar e documentar colunas existentes
print("Colunas mantidas no dataset:", EDA_humanresources_employee.columns.tolist())


Colunas mantidas no dataset: ['birthdate', 'businessentityid', 'currentflag', 'gender', 'hiredate', 'jobtitle', 'loginid', 'maritalstatus', 'modifieddate', 'nationalidnumber', 'organizationnode', 'rowguid', 'salariedflag', 'sickleavehours', 'vacationhours', 'hire_year']


In [27]:
for col in EDA_humanresources_employee.columns:
    print(f"Valores únicos em '{col}':", EDA_humanresources_employee[col].unique()[:3])  # Limitar a exibição a 10 valores



Valores únicos em 'birthdate': <DatetimeArray>
['1969-01-29 00:00:00', '1971-08-01 00:00:00', '1974-11-12 00:00:00']
Length: 3, dtype: datetime64[ns]
Valores únicos em 'businessentityid': [1 2 3]
Valores únicos em 'currentflag': [ True]
Valores únicos em 'gender': ['M' 'F']
Valores únicos em 'hiredate': <DatetimeArray>
['2009-01-14 00:00:00', '2008-01-31 00:00:00', '2007-11-11 00:00:00']
Length: 3, dtype: datetime64[ns]
Valores únicos em 'jobtitle': ['Chief Executive Officer' 'Vice President Of Engineering'
 'Engineering Manager']
Valores únicos em 'loginid': ['adventure-works\\ken0' 'adventure-works\\terri0'
 'adventure-works\\roberto0']
Valores únicos em 'maritalstatus': ['S' 'M']
Valores únicos em 'modifieddate': <DatetimeArray>
['2014-06-30 00:00:00', '2008-12-01 00:00:00', '2009-02-28 00:00:00']
Length: 3, dtype: datetime64[ns]
Valores únicos em 'nationalidnumber': ['295847284' '245797967' '509647174']
Valores únicos em 'organizationnode': ['/' '/1/' '/1/1/']
Valores únicos em 'ro

In [28]:
# Listar colunas binárias esperadas
binary_columns = ['currentflag', 'salariedflag']

# Verificar valores únicos em colunas binárias
for col in binary_columns:
    unique_values = EDA_humanresources_employee[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}")

   



Valores únicos em 'currentflag': [ True]
Valores únicos em 'salariedflag': [ True False]


In [29]:
# Contar valores em 'currentflag' e 'salariedflag'
print("Distribuição de 'currentflag':")
print(EDA_humanresources_employee['currentflag'].value_counts())

print("\nDistribuição de 'salariedflag':")
print(EDA_humanresources_employee['salariedflag'].value_counts())


#se vale a pena deletar ou não a coluna currentflag, já que só tem 1 valor e é true ?!

Distribuição de 'currentflag':
currentflag
True    290
Name: count, dtype: int64

Distribuição de 'salariedflag':
salariedflag
False    238
True      52
Name: count, dtype: int64


In [30]:
# 1. Verificar se todos os funcionários ativos têm currentflag = True, pois deveria ser false = demitido/desligado
print("Funcionários ativos errados:", EDA_humanresources_employee[EDA_humanresources_employee['currentflag'] != True])

# 2. Validar datas
print("Contratações futuras:", EDA_humanresources_employee[EDA_humanresources_employee['hiredate'] > pd.Timestamp.now()])
print("Modifieddate antes de hiredate:", EDA_humanresources_employee[EDA_humanresources_employee['modifieddate'] < EDA_humanresources_employee['hiredate']])

# 3. Verificar unicidade de identificadores
print("Duplicados em 'businessentityid':", EDA_humanresources_employee['businessentityid'].duplicated().sum())
print("Duplicados em 'nationalidnumber':", EDA_humanresources_employee['nationalidnumber'].duplicated().sum())



Funcionários ativos errados: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []
Contratações futuras: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []
Modifieddate antes de hiredate: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []
Duplicados em 'businessentityid': 0
Duplicados em 'nationalidnumber': 0


In [31]:
# Definir Dataset e Nome da Tabela
dataset_id = 'raw_data_cleaned'  # Substitua pelo nome do seu Dataset
table_id = 'humanresources_employee'   # Substitua pelo nome da sua Tabela
full_table_id = f"{client.project}.{dataset_id}.{table_id}"  # Nome completo da tabela

# Exportar para o BigQuery
EDA_humanresources_employee.to_gbq(full_table_id, if_exists='replace', credentials=client._credentials)




  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 8224.13it/s]

## ESTATÍSTICA DESCRITIVA

In [32]:
# Selecionar colunas relevantes para análise descritiva
cols_para_analise = ['sickleavehours', 'vacationhours', 'salariedflag']

# Garantir que as datas estejam no formato correto
EDA_humanresources_employee['hire_year'] = pd.to_datetime(EDA_humanresources_employee['hiredate']).dt.year

# Adicionar a nova coluna à lista
cols_para_analise.append('hire_year')

# Gerar estatísticas descritivas
analise_descritiva = EDA_humanresources_employee[cols_para_analise].describe(include='all')

# Substituir NaN por '-'
analise_descritiva = analise_descritiva.fillna('-')

print(analise_descritiva)


       sickleavehours vacationhours salariedflag hire_year
count          290.00        290.00          290    290.00
unique              -             -            2         -
top                 -             -        False         -
freq                -             -          238         -
mean            45.31         50.61            -   2009.02
std             14.54         28.79            -      1.01
min             20.00          0.00            -   2006.00
25%             33.00         26.25            -   2008.00
50%             46.00         51.00            -   2009.00
75%             58.00         75.00            -   2009.00
max             80.00         99.00            -   2013.00
